<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 3 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Student 1:</b> THAN Thanh An  
<b> Student 2:</b> NGUYEN Minh Duc
 
 
The aim of this session is to practice with Convolutional Neural Networks. Answers and experiments should be made by groups of one or two students. Each group should fill and run appropriate notebook cells. 


Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an pdf document using print as PDF (Ctrl+P). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by May 29th 2017.

Send you pdf file to benoit.huet@eurecom.fr and olfa.ben-ahmed@eurecom.fr using **[DeepLearning_lab2]** as Subject of your email.

# Introduction

In the last Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%.  Can  you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks:  **LeNet-5** to go to  more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell above to load the MNIST data that comes  with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels

print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example :
**y=softmax(Wx+b)** seen in the DeepLearing course last week. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the tensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to  visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
from __future__ import print_function
import tensorflow as tf


#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 100
batch_size = 10000
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()


#STEP 2 


# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
#             print(type(batch_xs))
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))




Epoch:  01   =====> Loss= 2.281444597
Epoch:  02   =====> Loss= 2.229430199
Epoch:  03   =====> Loss= 2.179733610
Epoch:  04   =====> Loss= 2.132062387
Epoch:  05   =====> Loss= 2.086399126
Epoch:  06   =====> Loss= 2.042266178
Epoch:  07   =====> Loss= 1.999297953
Epoch:  08   =====> Loss= 1.958605456
Epoch:  09   =====> Loss= 1.919204998
Epoch:  10   =====> Loss= 1.881934905
Epoch:  11   =====> Loss= 1.844592929
Epoch:  12   =====> Loss= 1.809497356
Epoch:  13   =====> Loss= 1.775510406
Epoch:  14   =====> Loss= 1.742494106
Epoch:  15   =====> Loss= 1.711519384
Epoch:  16   =====> Loss= 1.680916739
Epoch:  17   =====> Loss= 1.652868390
Epoch:  18   =====> Loss= 1.624090815
Epoch:  19   =====> Loss= 1.596414638
Epoch:  20   =====> Loss= 1.569908214
Epoch:  21   =====> Loss= 1.545104074
Epoch:  22   =====> Loss= 1.522130179
Epoch:  23   =====> Loss= 1.498071003
Epoch:  24   =====> Loss= 1.476168418
Epoch:  25   =====> Loss= 1.453928041
Epoch:  26   =====> Loss= 1.432838893
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Go to the **TP2** folder, 
- Open a Terminal and run the command line **"tensorboard --logdir= log_files/"**, it will generate an http link ,ex http://666.6.6.6:6006,
- Copy this  link into your web browser 


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

One you are now familar with **tensorFlow** and **tensorBoard**, you are in this section to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

In more advanced step you will make some optimizations to get more than 99% of accuracy. The best model can get to over 99.7% accuracy! 

For more information, have a look at this list of results : http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet 5 </span></center>





The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1: Convolutional.** The output shape should be 28x28x6 **Activation.** sigmoid **Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16. **Activation.** sigmoid **Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use **flatten*  from tensorflow.contrib.layers import flatten

**Layer 3: Fully Connected.** This should have 120 outputs. **Activation.** sigmoid

**Layer 4: Fully Connected.** This should have 84 outputs. **Activation.** sigmoid

**Layer 5: Fully Connected.** This should have 10 outputs. **Activation.** softmax


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [3]:
import numpy as np

In [4]:
#Helper functions  for weigths and bias initilization 

def weight_variable(shape):
  initial = tf.truncated_normal(shape, mean=0, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.01, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='VALID')



In [17]:
from tensorflow.contrib.layers import flatten

def LeNet5_Model(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = bias_variable([6])
    conv1_pre   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1_ac = tf.nn.sigmoid(conv1_pre)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1_ac, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = bias_variable([16])
    conv2_pre   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2_ac = tf.nn.sigmoid(conv2_pre)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2_ac, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1_pre   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1   = tf.nn.sigmoid(fc1_pre)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = bias_variable([84])
    fc2_pre    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.sigmoid(fc2_pre)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 10), mean = mu, stddev = sigma))
    fc3_b  = bias_variable([10])
    fc3_pre = tf.matmul(fc2, fc3_W) + fc3_b
    
    
    logits = fc3_pre
    return logits

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

The number of parameter of the model is the number of connection in the neural network. However, this neural network have convolution layers which has weight sharing mechanism so the true number of parameter is much lower than a fully connected neural network.

In [100]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    variable_parametes = 1
    for dim in shape:
        variable_parametes *= dim.value
    total_parameters += variable_parametes
print(total_parameters)

2105854


<b> Question 2.1.3. </b>  Start the training with the parameters cited below:

     Learning rate =0.1
     Loss Fucntion : Cross entropy
     Optimisateur: SGD
     Number of training iterations= 10000
     The batch size =128

In [ ]:
learning_rate = 0.1
training_epochs = 10000
batch_size = 128

In [18]:
# Parameters
learning_rate = 0.1
training_epochs = 100
batch_size = 128

x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
x_new = tf.map_fn(lambda x1: tf.image.pad_to_bounding_box(x1, 2, 2, 32, 32), x)
# x_normalize = tf.divide(x_new, 255.0)

# Defined model
model = LeNet5_Model(x_new)

# cost function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y)
loss_operation = tf.reduce_mean(cross_entropy)

# Optimization 
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_operation = optimizer.minimize(loss_operation)

In [21]:
from sklearn.utils import shuffle
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    for i in range(training_epochs):
        X_train, y_train = shuffle(X_train, y_train)
        
        for offset in range(0, num_examples, batch_size):
            end = offset + batch_size
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            batch_x = batch_x.reshape([-1,28,28,1])
            
            # start session
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation, sess)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        
    saver.save(sess, 'lenet')
    print("Model saved")

Training...
EPOCH 1 ...
Validation Accuracy = 0.110
EPOCH 2 ...
Validation Accuracy = 0.113
EPOCH 3 ...
Validation Accuracy = 0.113
EPOCH 4 ...
Validation Accuracy = 0.113
EPOCH 5 ...
Validation Accuracy = 0.113
EPOCH 6 ...
Validation Accuracy = 0.098
EPOCH 7 ...
Validation Accuracy = 0.168
EPOCH 8 ...
Validation Accuracy = 0.475
EPOCH 9 ...
Validation Accuracy = 0.750
EPOCH 10 ...
Validation Accuracy = 0.838
EPOCH 11 ...
Validation Accuracy = 0.888
EPOCH 12 ...
Validation Accuracy = 0.910
EPOCH 13 ...
Validation Accuracy = 0.927
EPOCH 14 ...
Validation Accuracy = 0.935
EPOCH 15 ...
Validation Accuracy = 0.944
EPOCH 16 ...
Validation Accuracy = 0.951
EPOCH 17 ...
Validation Accuracy = 0.955
EPOCH 18 ...
Validation Accuracy = 0.957
EPOCH 19 ...
Validation Accuracy = 0.962
EPOCH 20 ...
Validation Accuracy = 0.963
EPOCH 21 ...
Validation Accuracy = 0.965
EPOCH 22 ...
Validation Accuracy = 0.966
EPOCH 23 ...
Validation Accuracy = 0.969
EPOCH 24 ...
Validation Accuracy = 0.971
EPOCH 25 ...


KeyboardInterrupt: 

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [19]:
correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(xTest, yTest, sess):
    xTest = xTest.reshape([-1,28,28,1])
    num_examples = len(xTest)
    total_accuracy = 0
    
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = xTest[offset:offset+batch_size], yTest[offset:offset+batch_size]
        
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    
    return total_accuracy / num_examples

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [96]:
# Initializing the variables
import time

x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
keep_prob = tf.placeholder(tf.float32)

class NeuralNetwork:
    # the structure of this network is already defined
    # param to change: isdropout, 
#  Learning rate =0.1
#  Loss Fucntion: Cross entropy
#  Optimisateur: SGD
#  Number of training iterations= 10000
#  The batch size =128
    def __init__(self, LR=0.1, Iter=100, Batchsize=128, Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=False, Activation=tf.nn.sigmoid):
        self.logs_path = "log_files/"
        self.display_step = 1
        self.LR = LR
        self.Iter = Iter
        self.Batchsize = Batchsize
        self.Optimizer = Optimizer
        self.IsDropOut = IsDropOut
        self.Activation = Activation
        
        
        # x_normalize = tf.divide(x_new, 255.0)
#         cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y)
#         loss_operation = tf.reduce_mean(cross_entropy)
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
#         training_operation = optimizer.minimize(loss_operation)
        
        
        with tf.name_scope('Model'):
            # Model
            self.model = self.define_model(x, y)
        with tf.name_scope('Loss'):
            # Minimize error using cross entropy
            self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.model, labels=y)
            self.cost = tf.reduce_mean(self.cross_entropy)
        with tf.name_scope('Optimizer'):
            # Gradient Descent
            self.tfoptimizer = self.Optimizer(self.LR).minimize(self.cost)
        with tf.name_scope('Accuracy'):
            # Accuracy
            self.correct_prediction = tf.equal(tf.argmax(self.model, 1), tf.argmax(y, 1))
            self.accuracy_operation = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
            self.saver = tf.train.Saver()

        self.init = tf.global_variables_initializer()
        # Create a summary to monitor cost tensor
        self.summary_cost_epoch = tf.summary.scalar("Loss", self.cost)
        self.summary_cost_batch = tf.summary.scalar("Loss", self.cost)
        # Create a summary to monitor accuracy tensor
        self.summary_train_acc = tf.summary.scalar("Train_Accuracy", self.accuracy_operation)
        self.summary_validation_acc = tf.summary.scalar("Validation_Accuracy", self.accuracy_operation)
        # Merge all summaries into a single op
        
    
    
    def define_model(self, x, y):
        mu = 0
        sigma = 0.1
        # resized image input 28x28x1 => 32x32x1
        x_new = tf.map_fn(lambda x1: tf.image.pad_to_bounding_box(x1, 2, 2, 32, 32), x)
        
        # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
        conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
        conv1_b = bias_variable([6])
        conv1_pre   = tf.nn.conv2d(x_new, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

        # SOLUTION: Activation.
        conv1_ac = self.Activation(conv1_pre)

        # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
        conv1 = tf.nn.max_pool(conv1_ac, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

        # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
        conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
        conv2_b = bias_variable([16])
        conv2_pre   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b

        # SOLUTION: Activation.
        conv2_ac = self.Activation(conv2_pre)

        # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
        conv2 = tf.nn.max_pool(conv2_ac, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

        # SOLUTION: Flatten. Input = 5x5x16. Output = 400.t multiple values for keyword argument 'LR
        fc0   = flatten(conv2)

        # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
        fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
        fc1_b = tf.Variable(tf.zeros(120))
        fc1_pre   = tf.matmul(fc0, fc1_W) + fc1_b

        # SOLUTION: Activation.
        fc1   = self.Activation(fc1_pre)
        
        fc1_out = None
        # keep probability number to be input in training
        # drop out
        if (self.IsDropOut):
            fc1_out = tf.nn.dropout(fc1, keep_prob)
        else:
            fc1_out = fc1

        # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
        fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
        fc2_b  = bias_variable([84])
        fc2_pre = tf.matmul(fc1_out, fc2_W) + fc2_b

        # SOLUTION: Activation.
        fc2    = self.Activation(fc2_pre)
        
        fc2_out = None
        if (self.IsDropOut):
            fc2_out = tf.nn.dropout(fc2, keep_prob)
        else:
            fc2_out = fc2
            
        # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
        fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 10), mean = mu, stddev = sigma))
        fc3_b  = bias_variable([10])
        fc3_pre = tf.matmul(fc2_out, fc3_W) + fc3_b

        logits = fc3_pre
        return logits
        
    def evaluation(self, X_val, y_val, sess, acc_summary=None):
        X_val_reshaped = X_val.reshape([-1, 28, 28, 1])
        accuracy = 0
        if (acc_summary != None):
            if (self.IsDropOut):
                accuracy, summary_ = sess.run([self.accuracy_operation, acc_summary], feed_dict={x: X_val_reshaped, y: y_val, keep_prob: 1.0})
            else:
                accuracy, summary_ = sess.run([self.accuracy_operation, acc_summary], feed_dict={x: X_val_reshaped, y: y_val})
            return accuracy, summary_
        else:
            if (self.IsDropOut):
                accuracy = sess.run([self.accuracy_operation], feed_dict={x: X_val_reshaped, y: y_val, keep_prob: 1.0})
            else:
                accuracy = sess.run([self.accuracy_operation], feed_dict={x: X_val_reshaped, y: y_val})
            return accuracy
        
    def train(self, X_train, y_train, X_validation, y_validation):
        # Initializing the session 
        print ("Start Training!")
        ####
#         X_train, y_train = shuffle(X_train, y_train)
#         for offset in range(0, num_examples, batch_size):
#             end = offset + batch_size
#             batch_x, batch_y = X_train[offset:end], y_train[offset:end]
#             batch_x = batch_x.reshape([-1,28,28,1])
            
            
#             sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
# #         validation_accuracy = evaluate(X_validation, y_validation, sess)
#         validation_accuracy = evaluate(X_validation, y_validation, sess)
#         print("EPOCH {} ...".format(i+1))
#         print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        # Launch the graph for training
        with tf.Session() as sess:
            sess.close()
            sess.run(self.init)
            # op to write logs to Tensorboard
            summary_writer = tf.summary.FileWriter(self.logs_path, graph=tf.get_default_graph())
            # Training cycle
            for epoch in np.arange(self.Iter):
                avg_cost = 0.
                total_batch = int(mnist.train.num_examples/batch_size)
                X_train, y_train = shuffle(X_train, y_train)
                
                # Loop over all batches
                for offset in range(0, num_examples, batch_size):
                    end = offset + batch_size
                    batch_x, batch_y = X_train[offset:end], y_train[offset:end]
                    batch_x = batch_x.reshape([-1,28,28,1])
                    # Run optimization op (backprop), cost op (to get loss value)
                    # and summary nodes
        #             print(type(batch_xs))
                    c = None
                    if (self.IsDropOut == False):
                        _, c = sess.run([self.tfoptimizer, self.cost], 
                                                 feed_dict={x: batch_x, y: batch_y})
                    else:
                        _, c = sess.run([self.tfoptimizer, self.cost], 
                                                 feed_dict={x: batch_x, y: batch_y, keep_prob: 0.75})
                    # Write logs at every iteration
#                     summary_writer.add_summary(summary, epoch * total_batch + i)
                    # fix this
                    # Compute average loss
#                     print("Batch ", offset / batch_size + 1 , " loss: ", c)
                    avg_cost += c / total_batch
                # Display logs per epoch step
                
                if (epoch+1) % self.display_step == 0:
                    accu, accu_sum = self.evaluation(X_validation, y_validation, sess, self.summary_validation_acc)
                    print("Epoch: ", '%02d' % (epoch+1), 
                          ", Loss=", "{:.9f}".format(avg_cost), 
#                           ", Training accuracy=", self.evaluation(X_train, y_train, sess),
                          ", Validation accuracy=", accu
                         )
                    summary_writer.add_summary(accu_sum, epoch)
            
            print("Optimization Finished!")
            # Save model
            self.saver.save(sess, "mnist_nn_" + str(self.Iter) + "_" + str(time.time()), global_step=100)
            # Test model
            # Calculate accuracy
            print("Accuracy on test data: ", self.evaluation(mnist.test.images, mnist.test.labels, sess, None))
    

In [61]:
mnist_nn = NeuralNetwork(LR=0.1, Iter=100, Batchsize=128, 
                         Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=False, Activation=tf.nn.sigmoid)
mnist_nn.train(X_train, y_train, X_validation, y_validation)

Start Training!
Epoch:  01 , Loss= 2.310846389 , Training accuracy= 0.103909 , Validation accuracy= 0.11
Epoch:  02 , Loss= 2.310091832 , Training accuracy= 0.0984909 , Validation accuracy= 0.1002
Epoch:  03 , Loss= 2.310317077 , Training accuracy= 0.103909 , Validation accuracy= 0.11
Epoch:  04 , Loss= 2.309120519 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  05 , Loss= 2.308010817 , Training accuracy= 0.123891 , Validation accuracy= 0.1334
Epoch:  06 , Loss= 2.306360165 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  07 , Loss= 2.302617693 , Training accuracy= 0.103909 , Validation accuracy= 0.11
Epoch:  08 , Loss= 2.280734985 , Training accuracy= 0.2472 , Validation accuracy= 0.2612
Epoch:  09 , Loss= 2.005129505 , Training accuracy= 0.442091 , Validation accuracy= 0.4588
Epoch:  10 , Loss= 1.367010500 , Training accuracy= 0.717145 , Validation accuracy= 0.7308
Epoch:  11 , Loss= 0.780713167 , Training accuracy= 0.814055 , Validation accurac

KeyboardInterrupt: 

<b> Question 2.1.6 </b> : Use tensorBoard to visualise and save the LeNet5 Graph and all learning curves. 
Save all obtained figures in the folder **"TP2/MNIST_99_Challenge_Figures"**

In [ ]:
#  insert your obtained figure here 

In [ ]:
# your answer goas here

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>  Change the sigmoid function with a Relu :

- Retrain your network with SGD and AdamOptimizer and then fill the table above  :


| Optimizer            |  Gradient Descent         |AdamOptimizer |
| -------------        |: -------------: | ---------:   
| Validation Accuracy  |         |    |      
| Testing Accuracy     |           |    |       
| Training Time        |           |        |  |  


- Try with different learning rates for each Optimizer (0.0001 and 0.001 ) and different Batch sizes (50 and 128) for 20000 Epochs. 

- For each optimizer, plot (on the same curve) the **testing accuracies** function to **(learning rate, batch size)** 



- Did you reach the 99% accuracy ? What are the optimal parametres that gave you the best results? 








In [98]:
# your answer goas here

mnist_relu_001_50 = NeuralNetwork(LR=0.001, Iter=100, Batchsize=50, 
                         Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_001_50.train(X_train, y_train, X_validation, y_validation)

mnist_relu_0001_50 = NeuralNetwork(LR=0.0001, Iter=100, Batchsize=50, 
                         Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_0001_50.train(X_train, y_train, X_validation, y_validation)


mnist_relu_001_128 = NeuralNetwork(LR=0.001, Iter=100, Batchsize=128, 
                         Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_001_128.train(X_train, y_train, X_validation, y_validation)

mnist_relu_0001_128 = NeuralNetwork(LR=0.0001, Iter=100, Batchsize=128, 
                         Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_0001_128.train(X_train, y_train, X_validation, y_validation)


mnist_relu_001_50 = NeuralNetwork(LR=0.001, Iter=100, Batchsize=50, 
                         Optimizer=tf.train.AdamOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_001_50.train(X_train, y_train, X_validation, y_validation)

mnist_relu_0001_50 = NeuralNetwork(LR=0.0001, Iter=100, Batchsize=50, 
                         Optimizer=tf.train.AdamOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_0001_50.train(X_train, y_train, X_validation, y_validation)


mnist_relu_001_128 = NeuralNetwork(LR=0.001, Iter=100, Batchsize=128, 
                         Optimizer=tf.train.AdamOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_001_128.train(X_train, y_train, X_validation, y_validation)

mnist_relu_0001_128 = NeuralNetwork(LR=0.0001, Iter=100, Batchsize=128, 
                         Optimizer=tf.train.AdamOptimizer, IsDropOut=False, Activation=tf.nn.relu)
mnist_relu_0001_128.train(X_train, y_train, X_validation, y_validation)

Start Training!
Epoch:  01 , Loss= 2.298147947 , Validation accuracy= 0.1476
Epoch:  02 , Loss= 2.276781329 , Validation accuracy= 0.275
Optimization Finished!
Accuracy on test data:  [0.29350001]


<b> Question 2.2.2 </b>  What about applying a dropout layer on the Fully conntected layer and then retraining the model with the best Optimizer and parameters(Learning rate and Batsh size) obtained in *Question 2.2.1*  ? (probability to keep units=0.75). For this stage ensure that the keep prob is set to 1.0 to evaluate the 
performance of the network including all nodes.

In [66]:
# your implementaion goas here
mnist_dropout = NeuralNetwork(LR=0.1, Iter=100, Batchsize=128, 
                         Optimizer=tf.train.GradientDescentOptimizer, IsDropOut=True, Activation=tf.nn.sigmoid)
mnist_dropout.train(X_train, y_train, X_validation, y_validation)

Start Training!
Epoch:  01 , Loss= 2.325644633 , Training accuracy= 0.0964909 , Validation accuracy= 0.107
Epoch:  02 , Loss= 2.315682795 , Training accuracy= 0.0984909 , Validation accuracy= 0.1002
Epoch:  03 , Loss= 2.312604411 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  04 , Loss= 2.311279979 , Training accuracy= 0.103909 , Validation accuracy= 0.11
Epoch:  05 , Loss= 2.309775676 , Training accuracy= 0.0994545 , Validation accuracy= 0.0976
Epoch:  06 , Loss= 2.309114740 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  07 , Loss= 2.309088602 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  08 , Loss= 2.308952111 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  09 , Loss= 2.308398287 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  10 , Loss= 2.307952179 , Training accuracy= 0.112345 , Validation accuracy= 0.1126
Epoch:  11 , Loss= 2.307949162 , Training accuracy= 0.112345 , Validation 

KeyboardInterrupt: 

In [ ]:
Your comments go here